In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


In [2]:
path = r'C:\Users\fajar\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB043913,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284073,Frau,Diana,Pateman,Female,...,2024-09-05 00:00:00,7,WYNDHAM,WYNDHAM,WYNDHAM,1,0,0,Email and Phone,NO
1,KK058992,Karma Resorts,Other,Past Guests,Karma Kandara,PF281705,Mr,Paul,Guy,Male,...,2024-09-06 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
2,KK058308,Karma Resorts,Other,Past Guests,Karma Kandara,PF280280,Mrs,Marie,Sankoh,Female,...,2024-09-06 00:00:00,6,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
3,KK053240,Karma Resorts,Other,Past Guests,Karma Kandara,PF268349,Mrs,Marino,Stephanie Margaret,Female,...,2024-09-01 00:00:00,18,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
4,KK058837,Karma Resorts,Other,Past Guests,Karma Kandara,PF281403,Ms,Valerie Sophie,Bass,Female,...,2024-09-02 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,3,1,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,KLM002660,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF247202,<NA>,Sharon,Owens,Unknown,...,2024-09-09 00:00:00,1,DIR,OTA,RETAIL,2,0,0,Phone Only,NO
96,KLM004577,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF263810,<NA>,John,Walker,Unknown,...,2024-09-01 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
97,KSM011112,Karma Resorts,Other,Past Guests,Karma St Martins,PF248479,<NA>,Penny,Porter,Unknown,...,2024-09-02 00:00:00,5,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
98,KK048541,Karma Resorts,Other,Past Guests,Karma Kandara,PF259965,Mr,Dylan,Bailey,Male,...,2024-09-08 00:00:00,3,DIR,CHAT,DIGITAL,2,0,0,Email and Phone,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

In [7]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
21,KB043634,Karma Resorts,Other,Past Guests,Karma Bavaria,PF281149,Herr,Bernd,Gönner,Male,...,2024-09-01 00:00:00,4,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
22,KB043637,Karma Resorts,Other,Past Guests,Karma Bavaria,PF281149,Herr,Bernd,Gönner,Male,...,2024-09-01 00:00:00,4,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
31,KB044197,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284585,Herr,Michael,Lobermeier,Male,...,2024-09-04 00:00:00,2,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
33,KB041530,Karma Resorts,Other,Past Guests,Karma Bavaria,PF268902,Frau,Nina,Hennig,Female,...,2024-09-01 00:00:00,1,OTA,OTA,RETAIL,6,0,0,Email and Phone,NO
39,KB044193,Karma Resorts,Other,Past Guests,Karma Bavaria,PF270292,Herr,Klaus Andreas,Span,Male,...,2024-09-05 00:00:00,4,COMP,TEL,RETAIL,2,0,0,Email and Phone,NO
51,KK055225,Karma Resorts,Other,Past Guests,Karma Kandara,PF272380,Ms,Daniela,Georgi,Female,...,2024-09-04 00:00:00,3,OTA,OTA,RETAIL,2,1,0,Phone Only,NO
52,KB043555,Karma Resorts,Other,Past Guests,Karma Bavaria,PF267462,empty,Stefanie,Maier,Unknown,...,2024-09-01 00:00:00,1,OTA,OTA,RETAIL,3,0,0,Phone Only,NO
67,KSM011879,Karma Resorts,Other,Past Guests,Karma St Martins,PF263002,Miss,Anna,Davies,Unknown,...,2024-09-02 00:00:00,3,DIR,WEBSITE,DIGITAL,1,0,0,Email and Phone,NO
69,KLM003652,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF254426,Mrs,Christine,Cameron,Female,...,2024-09-01 00:00:00,1,DIR,WALKIN,RETAIL,2,0,0,Email Only,NO
70,KLM003653,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF254426,Mrs,Christine,Cameron,Female,...,2024-09-01 00:00:00,1,DIR,WALKIN,RETAIL,2,0,0,Email Only,NO


In [8]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB043913,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284073,Frau,Diana,Pateman,Female,...,2024-09-05 00:00:00,7,WYNDHAM,WYNDHAM,WYNDHAM,1,0,0,Email and Phone,NO
1,KK058992,Karma Resorts,Other,Past Guests,Karma Kandara,PF281705,Mr,Paul,Guy,Male,...,2024-09-06 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
2,KK058308,Karma Resorts,Other,Past Guests,Karma Kandara,PF280280,Mrs,Marie,Sankoh,Female,...,2024-09-06 00:00:00,6,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
3,KK053240,Karma Resorts,Other,Past Guests,Karma Kandara,PF268349,Mrs,Marino,Stephanie Margaret,Female,...,2024-09-01 00:00:00,18,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
4,KK058837,Karma Resorts,Other,Past Guests,Karma Kandara,PF281403,Ms,Valerie Sophie,Bass,Female,...,2024-09-02 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,3,1,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,KLM002660,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF247202,empty,Sharon,Owens,Unknown,...,2024-09-09 00:00:00,1,DIR,OTA,RETAIL,2,0,0,Phone Only,NO
81,KLM004577,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF263810,empty,John,Walker,Unknown,...,2024-09-01 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
82,KSM011112,Karma Resorts,Other,Past Guests,Karma St Martins,PF248479,empty,Penny,Porter,Unknown,...,2024-09-02 00:00:00,5,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
83,KK048541,Karma Resorts,Other,Past Guests,Karma Kandara,PF259965,Mr,Dylan,Bailey,Male,...,2024-09-08 00:00:00,3,DIR,CHAT,DIGITAL,2,0,0,Email and Phone,NO


In [9]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [10]:
find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [11]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
22,KB043126,Karma Resorts,Other,Past Guests,Karma Bavaria,PF168470,Herr,Joachim,Schug,Male,...,2024-09-03 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
55,KB040125,Karma Resorts,Other,Past Guests,Karma Bavaria,PF110261,empty,Franz,Lindstroem,Male,...,2024-09-01 00:00:00,3,COMP,GER_OWE,COMP,2,2,0,Email and Phone,NO
67,KSM013773,Karma Resorts,Other,Past Guests,Karma St Martins,PF283587,Mrs,Jessica,Kelly,Unknown,...,2024-09-02 00:00:00,7,DIR,WALKIN,RETAIL,2,0,0,Email and Phone,NO


In [12]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [13]:
count_data_akhir = df_analytic.shape[0]

In [14]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB042880,Karma Resorts,Other,Past Guests,Karma Bavaria,PF276839,Herr,Ludwig,Haslbeck,Male,...,2024-09-01 00:00:00,1,DIR,LGR,GROUP,2,0,0,Email Only,NO
1,KB043319,Karma Resorts,Other,Past Guests,Karma Bavaria,PF279617,Frau,Magdalena,Pinzl,Female,...,2024-09-01 00:00:00,2,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
2,KB043270,Karma Resorts,Other,Past Guests,Karma Bavaria,PF279385,Frau,Andrea,Meyer,Female,...,2024-09-01 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
3,KB042251,Karma Resorts,Other,Past Guests,Karma Bavaria,PF273236,Mrs,Brendan,Goodison,Female,...,2024-09-01 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
4,KB041530,Karma Resorts,Other,Past Guests,Karma Bavaria,PF268902,Frau,Nina,Hennig,Female,...,2024-09-01 00:00:00,1,OTA,OTA,RETAIL,6,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,KB043975,Karma Resorts,Other,Past Guests,Karma Bavaria,PF283218,Mr,Gary,Pryor,Male,...,2024-09-08 00:00:00,7,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
78,KB041718,Karma Resorts,Other,Past Guests,Karma Bavaria,PF269983,empty,Norbert,Schuster,Unknown,...,2024-09-08 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
79,KB041680,Karma Resorts,Other,Past Guests,Karma Bavaria,PF269652,empty,Volkmar,Meyhofer,Unknown,...,2024-09-08 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
80,KLM006116,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF280926,Ms,Joann,Shann,Female,...,2024-09-08 00:00:00,1,DIR,WEBSITE,DIGITAL,2,0,0,Email Only,NO


In [15]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [16]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


In [17]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB042880,Karma Resorts,Other,Past Guests,Karma Bavaria,PF276839,Herr,Ludwig,Haslbeck,Male,...,2024-09-01,1,DIR,LGR,GROUP,2,0,0,Email Only,NO
1,KB043223,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284470,Frau,Katharina,Zeidler,Female,...,2024-09-01,1,DIR,TEL,RETAIL,4,0,0,Email and Phone,NO
2,KB042839,Karma Resorts,Other,Past Guests,Karma Bavaria,PF276559,Herr,Julian,Dullinger,Male,...,2024-09-01,1,DIR,LGR,GROUP,2,0,0,Email and Phone,NO
3,KB043958,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284488,Frau,Michaela,Tiefer,Female,...,2024-09-01,1,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
4,KB043638,Karma Resorts,Other,Past Guests,Karma Bavaria,PF281149,Herr,Bernd,Gönner,Male,...,2024-09-01,4,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,KB040098,Karma Resorts,Other,Past Guests,Karma Bavaria,PF262326,,Grit,Koch,Unknown,...,2024-09-08,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
78,KB043975,Karma Resorts,Other,Past Guests,Karma Bavaria,PF283218,Mr,Gary,Pryor,Male,...,2024-09-08,7,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
79,KB041718,Karma Resorts,Other,Past Guests,Karma Bavaria,PF269983,,Norbert,Schuster,Unknown,...,2024-09-08,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
80,KB041680,Karma Resorts,Other,Past Guests,Karma Bavaria,PF269652,,Volkmar,Meyhofer,Unknown,...,2024-09-08,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO


In [18]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 100
Jumlah Data Yang Duplicate : 15
Jumlah Data Setelah Hapus Duplicate : 85
Jumlah Data Yang Termasuk Member: 3
Jumlah Data Setelah Hapus Member : 82
Jumlah Data Akhir : 82


In [19]:
df_analytic.to_excel(r'C:\Users\fajar\Documents\Python\Data\final_analytics.xlsx', index=False)